In [1]:
#import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/teresaflicek/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


## Nasa Mars News

In [3]:
#url of nasa mars news page to be scraped
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

#retrieve page with splinter
browser.visit(news_url)

In [4]:
#create beautiful soup object
news_soup=bs(browser.html, 'lxml')

In [5]:
#scrape the latest news title
mars_news = news_soup.find('div', class_='list_text')
news_title = mars_news.find('div', class_='content_title').text
print(news_title)

5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover


In [6]:
#scrape the paragraph text
news_p = news_soup.find('div', class_='article_teaser_body').text
print(news_p)

The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


## JPL Mars Space Images - Featured Image

In [120]:
#urls for mars space images
jpl_url = 'https://www.jpl.nasa.gov'
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [121]:
#retrieve page with splinter
browser.visit(image_url)

In [122]:
#create beautiful soup object
img_soup=bs(browser.html, 'lxml')

In [123]:
#pull the featured image url from the carousel item button
image = img_soup.find('a', class_="button fancybox")["data-fancybox-href"]
image

'/spaceimages/images/mediumsize/PIA18245_ip.jpg'

In [124]:
#create featured url image link
featured_image_url = jpl_url + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18245_ip.jpg


## Mars Facts

In [199]:
#save the url for the mars facts table into a variable
facts_url = 'https://space-facts.com/mars/'

In [200]:
#read only the table information from the webpage
table = pd.read_html(facts_url)[0]
table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [201]:
#rename the columns
facts_df = table.rename(columns={0: "Description", 1: "Mars"})
facts_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [202]:
#set the index to description
facts_df.set_index("Description")

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [204]:
#read the table into an html string - return facts_df.to_html()
facts_html_table = facts_df.to_html("table.html")
facts_html_table

In [131]:
#strip unwanted new lines
facts_html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Mars</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronome

## Mars Hemispheres

In [162]:
#url for mars hemispheres
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [163]:
#retrieve page with splinter
browser.visit(hem_url)

In [185]:
#iterate through each link to find the title and image url
links=browser.find_by_css("a.product-item h3")

#empty list for hemisphere image urls dictionaries
hemisphere_image_urls = []

#for loop for all hemisphere links
for x in range(len(links)):
    mars = {}
    
    browser.find_by_css("a.product-item h3")[x].click()
    
    hem_soup=bs(browser.html, 'lxml')
    
    title = hem_soup.find("h2", class_="title").text
    img_url = hem_soup.find("img", class_="wide-image")['src']
    img_url_full = f"https://astrogeology.usgs.gov{img_url}"
    
    mars['title']= title
    mars['img_url'] = img_url_full
    
    hemisphere_image_urls.append(mars)
    
    browser.back()
browser.quit()
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
